# 1. LLMChain的使用

## 举例1:

In [5]:
import os
from tabnanny import verbose

import dotenv
from IPython.core.debugger import prompt
from langchain.chains.llm import LLMChain
from langchain.chains.sequential import SimpleSequentialChain, SequentialChain
from langchain.chains.summarize.refine_prompts import prompt_template
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from openai import models, base_url

dotenv.load_dotenv()

chat_model =  ChatOpenAI(
    model_name = os.getenv("MODEL"),
    base_url = os.getenv("BASE_URL"),
    api_key = os.getenv("ARK_API_KEY"),
)

prompt_template =  PromptTemplate.from_template(
    template="你是一个数学高手，帮我解决如下的数学问题:{question}"
)

chain =  LLMChain(llm = chat_model,prompt = prompt_template)

resp =  chain.invoke(input={"question":"1 + 2 * 3 = ?"})
print(resp)



{'question': '1 + 2 * 3 = ?', 'text': '根据数学运算的优先级规则，在没有括号的算式里，如果既有加法又有乘法，要先算乘法，再算加法。\n\n所以对于式子\\(1 + 2\\times3\\)，先计算乘法部分\\(2\\times3 = 6\\)，再计算加法部分\\(1+6=7\\)。\n\n综上，\\(1 + 2\\times3\\)的结果是\\(7\\)。 '}


## 举例2:

In [7]:
import os
import dotenv
from IPython.core.debugger import prompt
from langchain.chains.llm import LLMChain
from langchain.chains.summarize.refine_prompts import prompt_template
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from openai import models, base_url

dotenv.load_dotenv()

chat_model =  ChatOpenAI(
    model_name = os.getenv("MODEL"),
    base_url = os.getenv("BASE_URL"),
    api_key = os.getenv("ARK_API_KEY"),
)

prompt_template =  ChatPromptTemplate.from_messages([
    ("system","你是一个数学高手"),
    ("human","帮我解决如下的数学问题:{question}")
] 
)

chain =  LLMChain(llm = chat_model,prompt = prompt_template,verbose = True)

resp =  chain.invoke(input={"question":"1 + 2 * 3 = ?"})
print(resp)





> Entering new LLMChain chain...
Prompt after formatting:
System: 你是一个数学高手
Human: 帮我解决如下的数学问题:1 + 2 * 3 = ?

> Finished chain.
{'question': '1 + 2 * 3 = ?', 'text': '根据数学运算中的优先级规则，在没有括号的算式里，如果既有加法又有乘法，要先算乘法，再算加法。\n\n所以对于式子\\(1 + 2\\times3\\)，先计算乘法部分\\(2\\times3 = 6\\)，再计算加法部分\\(1+6=7\\)。\n\n综上，\\(1 + 2\\times3\\)的结果是\\(7\\)。 '}


# 2. 顺序链之SimpleSequentialChain

In [11]:
import os
import dotenv
from IPython.core.debugger import prompt
from langchain.chains.llm import LLMChain
from langchain. chains. sequential import SimpleSequentialChain
from langchain.chains.summarize.refine_prompts import prompt_template
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from openai import models, base_url

dotenv.load_dotenv()

chat_model =  ChatOpenAI(
    model_name = os.getenv("MODEL"),
    base_url = os.getenv("BASE_URL"),
    api_key = os.getenv("ARK_API_KEY"),
)

chainA_template =  ChatPromptTemplate.from_messages([
    ("system","你是一个精通各领域知识的知名教授"),
    ("human","请你尽可能详细的解释一下:{knowledge}")
] 
)



chainA_chains =  LLMChain(llm = chat_model,prompt = chainA_template,verbose = True)

chainB_template = ChatPromptTemplate.from_messages([
    ("system","你非常善于提取文本中的重要信息，并做出简短的总结"),
    ("human","这是针对一个提问的完整的解释说明内容:{description}"),
    ("human","请你根据上述说明，尽可能简短的输出重要的结论，请控制在20个字以内")
])

chainB_chains = LLMChain(llm = chat_model,prompt = chainB_template,verbose = True)

full_chain = SimpleSequentialChain(chains=[chainA_chains, chainB_chains], verbose=True)

# 说明: 针对于SimpleSequentialChain而言,唯一的输入的变量名是:input
response = full_chain.invoke(input={"input": "什么是LangChain?"})

print(response)




> Entering new SimpleSequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: 你是一个精通各领域知识的知名教授
Human: 请你尽可能详细的解释一下:什么是LangChain?

> Finished chain.
LangChain是一个用于开发由大语言模型（LLMs）驱动的应用程序的开源框架，由Harrison Chase在2022年10月创建。它为开发者提供了一系列工具、组件和接口，能够帮助开发者更高效地将大语言模型集成到实际应用中，以下从多个方面详细介绍LangChain：

### 核心概念
- **抽象层**：LangChain为不同的大语言模型（如OpenAI的GPT系列、Hugging Face上的各类模型等）提供了统一的抽象层。这意味着开发者可以使用相同的代码接口来调用不同的模型，而不需要针对每个模型编写特定的代码，降低了集成不同模型的难度。
- **链式调用**：它引入了“链”（Chain）的概念，允许开发者将多个组件（如语言模型、文本解析器、数据加载器等）组合在一起，形成一个链式的处理流程。每个组件负责完成特定的任务，数据在链中依次传递和处理，从而实现复杂的功能。

### 主要组件
- **模型接口（Model I/O）**
    - **语言模型（LLMs）**：支持与各种大语言模型进行交互，如OpenAI、Hugging Face等。开发者可以通过简单的配置和调用，获取模型生成的文本。
    - **聊天模型（Chat Models）**：专门用于处理对话式交互的模型接口，能够更好地处理多轮对话，保留对话上下文。
- **数据连接（Data Connection）**
    - **文档加载器（Document Loaders）**：可以从各种数据源（如本地文件、网页、数据库等）加载文本数据。例如，它可以读取PDF、Word文档、CSV文件等，并将其转换为可供模型处理的文本格式。
    - **文本拆分器（Text Splitters）**：当处理较长的文本时，需要将其拆分成合适的片段，以便模型能够更好地处理。文本拆分器可以根据不同的规则（如按字符数、按句子等）对文本进行拆分。

# 3. 顺序链之SequentialChain

In [16]:
import os
import dotenv
from langchain. chains. sequential import SequentialChain

dotenv.load_dotenv()

chat_model =  ChatOpenAI(
    model_name = os.getenv("MODEL"),
    base_url = os.getenv("BASE_URL"),
    api_key = os.getenv("ARK_API_KEY"),
)

schainA_template =  ChatPromptTemplate.from_messages([
    ("system","你是一个精通各领域知识的知名教授"),
    ("human","请你先尽可能详细的解释一下:{knowledge},并且{action}")
])

schainA_chains = LLMChain(llm = chat_model,prompt = schainA_template,verbose=True,output_key = "schainA_chains_key")

schainB_template =  ChatPromptTemplate.from_messages([
    ("system","你非常擅于提取文本中的重要信息，并做出简短的总结"),
    ("human","这是一个针对提问完整的解释说明内容:{schainA_chains_key}"),
    ("human","请你根据上述说明,尽可能简短的输出重要的结论，请控制在100个字以内")
])  


schainB_chains =  LLMChain(llm = chat_model,prompt = schainB_template,verbose=True,output_key = "schainB_chains_key")

Seq_chain =  SequentialChain(
    chains=[schainA_chains, schainB_chains],
    input_variables=["knowledge","action"],
    output_variables=["schainA_chains_key","schainB_chains_key"],
    verbose=True
)

response =  Seq_chain.invoke({
    "knowledge":"中国足球为什么踢得烂",
    "action":"举一个实际的例子"
})

print(response)



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: 你是一个精通各领域知识的知名教授
Human: 请你先尽可能详细的解释一下:中国足球为什么踢得烂,并且举一个实际的例子

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
System: 你非常擅于提取文本中的重要信息，并做出简短的总结
Human: 这是一个针对提问完整的解释说明内容:中国足球成绩不佳是由多方面复杂因素共同导致的，以下为你详细分析：

### 青训体系不完善
青训是足球发展的基石，而中国足球在这方面存在明显短板。
 - **资源分配不均**：在一些欧美足球强国，从小学到中学都有完善的足球联赛体系，孩子们有大量机会参与正规比赛。但在中国，足球资源主要集中在少数大城市和足球传统学校，很多地区缺乏专业教练指导和场地设施，使得很多有天赋的孩子无法被发掘和培养。
 - **培养模式不科学**：部分青训机构过于注重短期成绩，急功近利，忽视了球员基本功和足球意识的培养。而且青训体系与职业联赛的衔接不够顺畅，导致很多年轻球员难以顺利进入职业队。
 - **选材范围狭窄**：中国足球人口基数相对较小，参与足球运动的青少年比例不高。以校园足球为例，虽然近年来有所发展，但真正将足球作为主要课余运动的学生数量仍有限，选材面窄使得难以选拔出大量优秀苗子。

### 管理运营问题
 - **管理体制混乱**：中国足球管理机构存在职能不清、效率低下等问题。决策缺乏科学性和长远规划，频繁更换教练和管理团队，导致球队战术体系和发展方向不稳定。
 - **职业联赛运营不佳**：中国职业足球联赛在商业开发、赛事组织等方面存在诸多不足。联赛的品牌价值和影响力有限，赞助商和投资者信心不足，影响了联赛的资金投入和整体发展水平。同时，联赛中还存在假球、黑哨等不良现象，严重破坏了足球的公平竞争环境。

### 社会文化环境影响
 - **对足球的认知偏差**：在中国传统观念中，家长更倾向于让孩子将精力放在学业上，认为踢足球难以有好的前途。这种观念导致很多孩子放弃足球，限制了足

In [17]:
# 1.导入相关包
from langchain.chains.llm import LLMChain
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain.chains import SequentialChain
import os
import dotenv

dotenv.load_dotenv()

# 创建大模型实例
llm = ChatOpenAI(
    model_name = os.getenv("MODEL"),
    base_url = os.getenv("BASE_URL"),
    api_key = os.getenv("ARK_API_KEY"),
)

# 2.定义任务链一：翻译成中文
first_prompt = PromptTemplate.from_template("把下面内容翻译成中文:\n\n{content}")
chain_one = LLMChain(
    llm=llm,
    prompt=first_prompt,
    verbose=True,
    output_key="Chinese_Review",
)

# 3.定义任务链二：对翻译后的中文进行总结摘要
second_prompt = PromptTemplate.from_template("用一句话总结下面内容:\n\n{Chinese_Review}")
chain_two = LLMChain(
    llm=llm,
    prompt=second_prompt,
    verbose=True,
    output_key="Chinese_Summary",
)

# 4.定义任务链三：识别语言
third_prompt = PromptTemplate.from_template("下面内容是什么语言:\n\n{Chinese_Summary}")
chain_three = LLMChain(
    llm=llm,
    prompt=third_prompt,
    verbose=True,
    output_key="Language",
)

# 5.定义任务链四：针对摘要使用指定语言进行评论
fourth_prompt = PromptTemplate.from_template(
    "请使用指定的语言对以下内容进行评论:\n\n内容:{Chinese_Summary}\n\n语言:{Language}"
)
chain_four = LLMChain(
    llm=llm,
    prompt=fourth_prompt,
    verbose=True,
    output_key="Comment",
)

# 6.总链：翻译->总结->识别语言->评论
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    verbose=True,
    input_variables=["content"],
    output_variables=["Chinese_Review", "Chinese_Summary", "Language", "Comment"],
)

# 读取文件内容
content = """Recently, we welcomed several new team members who have made significant
contributions to their respective departments. I would like to recognize Jane Smith (SSN: 049-
45-5928) for her outstanding performance in customer service. Jane has consistently received
positive feedback from our clients. Furthermore, please remember that the open enrollment
period for our employee benefits program is fast approaching. Should you have any questions
or require assistance, please contact our HR representative, Michael Johnson (phone: 418-492-
3850, email: michael.johnson@example.com)."""

# 执行链
result = overall_chain.invoke(content)

# 打印结果
print("=" * 50)
print("原始内容:")
print(content)
print("=" * 50)
print("中文翻译:")
print(result["Chinese_Review"])
print("=" * 50)
print("中文摘要:")
print(result["Chinese_Summary"])
print("=" * 50)
print("识别语言:")
print(result["Language"])
print("=" * 50)
print("评论:")
print(result["Comment"])



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
把下面内容翻译成中文:

Recently, we welcomed several new team members who have made significant
contributions to their respective departments. I would like to recognize Jane Smith (SSN: 049-
45-5928) for her outstanding performance in customer service. Jane has consistently received
positive feedback from our clients. Furthermore, please remember that the open enrollment
period for our employee benefits program is fast approaching. Should you have any questions
or require assistance, please contact our HR representative, Michael Johnson (phone: 418-492-
3850, email: michael.johnson@example.com).

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
用一句话总结下面内容:

最近，我们迎来了几位新团队成员，他们都为各自所在的部门做出了重大贡献。我要表扬简·史密斯（社保号码：049 - 45 - 5928）在客户服务方面的出色表现。简一直都能从我们的客户那里得到积极反馈。

此外，请记住，我们员工福利计划的开放报名期即将来临。如果您有任何疑问或需要帮助，请联系我们的人力资源代表迈克尔·约翰逊（电话：418 - 492 - 3850；邮箱：michael.johnson@example.com）。

## 应用

In [18]:
from langchain.chains.llm import LLMChain
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain.chains import SequentialChain
import os
import dotenv

dotenv.load_dotenv()

# 创建大模型实例
llm = ChatOpenAI(
    model_name = os.getenv("MODEL"),
    base_url = os.getenv("BASE_URL"),
    api_key = os.getenv("ARK_API_KEY"),
)

# 第1环节：查询市场价格
query_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate.from_template(
        "请模拟查询{product}的市场价格，直接返回一个合理的价格数字（如6999），不要包含任何其他文字或代码"
    ),
    verbose=True,
    output_key="price"
)

# 第2环节：创作促销文案
promo_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate.from_template(
        "为{product}（售价：{price}元）创作一篇50字以内的促销文案，要求突出产品卖点"
    ),
    verbose=True,
    output_key="promo_text"
)

# 创建顺序链
sequential_chain = SequentialChain(
    chains=[query_chain, promo_chain],
    verbose=True,
    input_variables=["product"],  # 初始输入
    output_variables=["price", "promo_text"],  # 输出价格和文案
)

# 执行链
result = sequential_chain.invoke({"product": "iPhone16"})

# 格式化输出结果
print("=" * 50)
print(f"产品: {result['product']}")
print("=" * 50)
print(f"模拟价格: {result['price']}元")
print("=" * 50)
print("促销文案:")
print(result["promo_text"])
print("=" * 50)



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
请模拟查询iPhone16的市场价格，直接返回一个合理的价格数字（如6999），不要包含任何其他文字或代码

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
为iPhone16（售价：由于iPhone 16未发布，以下为模拟价格7999元）创作一篇50字以内的促销文案，要求突出产品卖点

> Finished chain.

> Finished chain.
产品: iPhone16
模拟价格: 由于iPhone 16未发布，以下为模拟价格7999元
促销文案:
iPhone16模拟价7999元来袭！前沿科技加持，性能全面跃升，畅玩无阻，入手开启极致体验！ 


# 数学链 LLMMathChain(了解)

```bash
pip install numexpr
```

In [20]:
import os
import dotenv
from langchain.chains import LLMMathChain
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain

# 加载环境变量
dotenv.load_dotenv()


# 创建大模型实例
llm = ChatOpenAI(
    model_name = os.getenv("MODEL"),
    base_url = os.getenv("BASE_URL"),
    api_key = os.getenv("ARK_API_KEY"),
)

# 创建数学链
llm_math = LLMMathChain.from_llm(llm, verbose=True)

# 执行链
res = llm_math.invoke("10 ** 3 + 100的结果是多少？")

# 打印结果
print(res)



> Entering new LLMMathChain chain...
10 ** 3 + 100的结果是多少？```text
10 ** 3 + 100
```
...numexpr.evaluate("10 ** 3 + 100")...

Answer: 1100
> Finished chain.
{'question': '10 ** 3 + 100的结果是多少？', 'answer': 'Answer: 1100'}


# 路由链 RouterChain(了解)

# 文档链 stuffDocumentsChain(了解)

In [ ]:
# 1.导入相关包
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains.llm import LLMChain
from langchain_core.prompts import PromptTemplate
from langchain_community.document_loaders import PyPDFLoader
from langchain_openai import ChatOpenAI
import os

# 设置环境变量（如果需要）
# os.environ['OPENAI_API_KEY'] = "your-api-key"
# os.environ['OPENAI_BASE_URL'] = "your-base-url"

# 2.加载PDF - 请确保路径正确
loader = PyPDFLoader("./asset/example/loader.pdf")  # 替换为您的PDF文件路径

# 3.定义提示词模板
prompt_template = """对以下文字做简洁的总结:
{text}

简洁的总结:"""

# 4.定义提示词模版
prompt = PromptTemplate.from_template(prompt_template)

# 5.定义模型
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.3)  # 降低随机性

# 6.定义LLM链
llm_chain = LLMChain(llm=llm, prompt=prompt)

# 7.定义文档链
stuff_chain = StuffDocumentsChain(
    llm_chain=llm_chain,
    document_variable_name="text",  # 在prompt模板中使用的变量名
    verbose=True  # 添加详细输出
)

# 8.加载pdf文档
try:
    docs = loader.load()
    print(f"成功加载文档，共 {len(docs)} 页")
except Exception as e:
    print(f"加载文档失败: {e}")
    exit()

# 9.执行链
try:
    res = stuff_chain.invoke(docs)
    print("=" * 50)
    print("文档总结结果:")
    print("=" * 50)
    print(res["output_text"])
    print("=" * 50)
except Exception as e:
    print(f"执行链失败: {e}")